<a href="https://colab.research.google.com/github/kev-in-xu/Phylo-Exceptions/blob/main/Cluster_Align_Ancestor_Sequence_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#$\color{#d7bb1b}{\text{Introduction}}$

This pipeline clusters fasta files to a desired number of sequences.

###  Sources:

"CD-HIT: a fast program for clustering and comparing large sets of protein or nucleotide sequences", Weizhong Li & Adam Godzik. Bioinformatics, (2006) 22:1658-1659

"CD-HIT: accelerated for clustering the next generation sequencing data", Limin Fu, Beifang Niu, Zhengwei Zhu, Sitao Wu & Weizhong Li. Bioinformatics, (2012) 28:3150-3152

R.C. Edgar (2021) "MUSCLE v5 enables improved estimates of phylogenetic tree confidence by ensemble bootstrapping"
https://www.biorxiv.org/content/10.1101/2021.06.20.449169v1.full.pdf

#$\color{#17bb1b}{\text{Setup: G-Drive, BioConda, File}}$

In [ ]:
# Storage path setup (Enter path of the directory you want to save to)

%env STORAGE_FILE_PATH=/content/drive/MyDrive/Greene-Lab/CAASS

In [ ]:
# Mounts Google Drive. (A pop-up window will appear to ask for permission)

from google.colab import drive
drive.mount('/content/drive')

! mkdir $STORAGE_FILE_PATH
! mkdir $STORAGE_FILE_PATH/input-files
! mkdir $STORAGE_FILE_PATH/cleaned-files
! mkdir $STORAGE_FILE_PATH/cluster-files
! mkdir $STORAGE_FILE_PATH/align-files
! mkdir $STORAGE_FILE_PATH/tree-files
! mkdir $STORAGE_FILE_PATH/ancestor-files

In [ ]:
# Installs Conda package manager.

! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
#@title File Upload
#@markdown <b>Input files should be unaligned fastas.</b>

#@markdown Auto upload allows upload of only one file at a time.
#@markdown A prompt will appear below this code block for you to select your file

auto_upload = True #@param {type:"boolean"}

if(auto_upload):
  from google.colab import files
  uploaded = files.upload()

  for fn in uploaded:
    !cp $fn $STORAGE_FILE_PATH/input-files $STORAGE_FILE_PATH/input-files
    %env INPUT=$fn
    !rm $fn

#@markdown If manual upload, upload your input files or into
#@markdown /CAASS/input-files on your drive and copy the file name below:

else:
  file_name = "" #@param {type:"string"}
  %env INPUT=$file_name

In [ ]:
! $fn

In [ ]:
# Initialization of file names for the rest of the pipeline

ext = %env INPUT
%env CLEANED=cleaned_$ext
%env CLUSTERED=clustered_$ext
%env ALIGNED=aligned_$ext
%env ANCESTRAL=ancestral_$ext
%env STRUCTURED=stuctural_$ext

tree_ext = ext[:ext.find(".")] + ".tre"
%env TREE=tree_$tree_ext

#$\color{#1bab8b}{\text{spaghetti: Clean-up}}$

In [ ]:
#@title Processing Options
# Initialization
path = %env STORAGE_FILE_PATH
input = %env INPUT
cleaned = %env CLEANED
outFile = path + "/cleaned-files/" + cleaned
inFile = path + "/input-files/" + input

# reads file into string array
text = ""
with open(inFile, "r") as readFile:
  for x in readFile:
    if bool(x.rstrip()):
      text += x
    else:
      print("empty")

lines = text.split("\n")

#@markdown Adds prefix to all sequence headers
#@markdown (useful when combining datasets with multiple proteins).
#@markdown [blank by default]
prefix = "" #@param {type: "string"}

for x in range(len(lines)):
  line = lines[x]
  try:
    if (line[0] == '>'):
      lines[x] = ">" + prefix + "_" + line[1:]
  except:
    print("endOfFile")


#@markdown Selecting delete_seqs removes sequences containing "X" or "-".
#@markdown
#@markdown Selecting clean_seqs keeps all sequences but deletes instances of "x" or "-".
delete_seqs = False #@param {type:"boolean"}
clean_seqs = False #@param {type:"boolean"}

index = 0
count = 0
new_lines = []

if (delete_seqs):
  for x in range(int(len(lines)/2)):
    header = lines[2*x]
    body = lines[2*x+1]
    if (body.find('X') == -1 and body.find('-') == -1):
      new_lines.append(header)
      new_lines.append(body)
    else:
      count += 1
  lines = new_lines
  print("deleted " + str(count) + " seqs for 'X' and '-'")

if (clean_seqs):
  for x in range(len(lines)):
    line = lines[x]
    try:
      if (line[0] != '>'):
        lines[x] = line.replace("X","").replace("-","")
    except:
      print("endOfFile")


#@markdown Selecting delete_low_q removes sequences labeled as "LOW-QUALITY" or
#@markdown are the secondary / tertiary isoforms of a given species' protein.
delete_low_q = False #@param {type:"boolean"}

index = 0
count = 0
new_lines = []

if(delete_low_q):
  for x in range(int(len(lines)/2)):
    header = lines[2*x]
    body = lines[2*x+1]
    if (header.find("X2") == -1 and header.find("X3") == -1 and header.find("LOW") == -1):
        new_lines.append(header)
        new_lines.append(body)
    else:
      count += 1

  lines = new_lines
  print("deleted " + str(count) + " low quality or secondary isoform seqs")

#@markdown Selecting lengthen_seqs lengthens human and yeast sequences. (this ensures
#@markdown they are representatives during the clustering process)

lengthen_seqs = False #@param {type:"boolean"}
index = 0
poly_A_tail = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"

if(lengthen_seqs):
  for x in range(int(len(lines)/2)):
    header = lines[2*x]
    body = lines[2*x+1]
    if (header.lower().find("homo sapiens") != -1 or
        header.lower().find("saccharomyces cerevisiae") != -1):
      lines[2*x] = header
      lines[2*x+1] = body + poly_A_tail


#@markdown Selecting concat_header ensures entire headers shows up during tree labeling
concat_header = True #@param {type:"boolean"}

if(concat_header):
  for x in range(len(lines)):
    if lines[x][0] == '>':
      lines[x] = lines[x].replace(" ","-").replace(":","").replace("(","").replace(")","")


#writes to file
with open(outFile, "w") as writeFile:
  for line in lines:
    writeFile.write(line + "\n")

#$\color{#2f7fcf}{\text{cd-hit: Clustering}}$

In [ ]:
# Installs cd-hit
a = ! conda list
if (a.nlstr.find("cd-hit") == -1):
  ! conda install -c bioconda cd-hit -y
else:
  print("cd-hit already installed")

## cd-hit runtime

Parameter setting for input and output files. The optimal similarity threshold will be found based on the maximum sequence number provided

In [ ]:
MAX_CLUSTERS = 10000
MIN_THRESHOLD = 0.7
MAX_THRESHOLD = 0.98

In [ ]:
# setup to allow next step
%env THRESHOLD = 0
def run():
  ! eval cd-hit -i $STORAGE_FILE_PATH/cleaned-files/$CLEANED -o $STORAGE_FILE_PATH/cluster-files/$CLUSTERED -c $THRESHOLD

In [ ]:
# captures console output while clustering at various thresholds
%%capture c
for i in range(int(MIN_THRESHOLD*100), int(MAX_THRESHOLD*100)+1):
  a = i/100 # tests thresholds from min to max with increment of 0.01
  %env THRESHOLD = $a
  run()

In [ ]:
# checks output for cluster number and finds optimal threshold
# then overwrites output file with optimal clustering

maxi = 0
b=0
for i in range(int((MAX_THRESHOLD-MIN_THRESHOLD)*100)):
  index = c.stdout.find("THRESHOLD",b)
  b = c.stdout.find("finished",index)
  numOutput = c.stdout[b+16:b+20]
  if (int(numOutput) < MAX_CLUSTERS):
    maxi = i # finds the most sensitive threshold that generates < MAX_CLUSTERS

optimum = MIN_THRESHOLD + maxi*0.01
%env THRESHOLD = $optimum

! eval cd-hit -i $STORAGE_FILE_PATH/cleaned-files/$CLEANED -o $STORAGE_FILE_PATH/cluster-files/$CLUSTERED -c $THRESHOLD -g 1

In [ ]:
! eval cd-hit -i $STORAGE_FILE_PATH/cleaned-files/$CLEANED -o $STORAGE_FILE_PATH/cluster-files/$CLUSTERED -c 0.98 -g 1

In [ ]:
#@title trim lengthened sequences
#@markdown if human and yeast sequences were lengthened in earlier steps,
#@markdown they need to be trimmed at this stage before alignment

#Initialization
clustered = %env CLUSTERED
clusterFile = path + "/cluster-files/" + clustered


trim_seqs = True #@param {type:"boolean"}

# reads file into string array
if(trim_seqs):
  text = ""
  with open(clusterFile, "r") as readFile:
    for x in readFile:
      if bool(x.rstrip()):
        text += x

  lines = text.split("\n")

  index = 0

  for x in range(int(len(lines)/2)):
    header = lines[2*x]
    body = lines[2*x+1]
    if ((header.lower().find("sapiens") != -1 or
        header.lower().find("cerevisiae") != -1) and
        body[-3:] == 'AAA'):
      lines[2*x] = header
      lines[2*x+1] = body[:-120] # removes the 120 A's that were added

  with open(clusterFile, "w") as writeFile:
    for line in lines:
      writeFile.write(line + "\n")

#$\color{#8b6bbb}{\text{MUSCLE5: Alignment}}$

In [ ]:
# Installs MUSCLE5

a = ! conda list
if (a.nlstr.find("muscle") == -1):
  ! conda install -c bioconda muscle -y
else:
  print("muscle already installed")

In [ ]:
# EXECUTE THIS IF RUNNING from a folder that's not aligned-files
! muscle -in $STORAGE_FILE_PATH/cleaned-files/$CLEANED -out $STORAGE_FILE_PATH/align-files/$ALIGNED

In [ ]:
# File Setup
ext = %env CLUSTERED
%env ALIGNED= aligned_$ext


# Run Command
! muscle -in $STORAGE_FILE_PATH/cluster-files/$CLUSTERED -out $STORAGE_FILE_PATH/align-files/$ALIGNED

#$\color{#bb5b8b}{\text{FastTree: Tree Building}}$

In [ ]:
# Installs FastTree
a = ! conda list
if (a.nlstr.find("fasttree") == -1):
  ! conda install -c bioconda fasttree -y
else:
  print("fasttree already installed")

In [ ]:
# File Setup (Tree will be generated in .tre format)
%env ALIGNED

# Run Command
! FastTree $STORAGE_FILE_PATH/align-files/$ALIGNED > $STORAGE_FILE_PATH/tree-files/$TREE

#Extra

###Getting status updates from FireProt-ASR

In [ ]:
%env ID = dm8y77_16
ID = %env ID

In [ ]:
import requests
import pprint as pp
site = 'https://loschmidt.chemi.muni.cz/fireprotasr/?action=calculation&job=' + ID + '&'
response = requests.get(site)
response.text
print(response.text)

## post processing

In [ ]:
import os
try:
  os.remove("/content/drive/MyDrive/cd-hit-trial/output-files/Copy of 300_clustered_Dmc1.fasta")
except:
  print("error while deleting file.")